In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.8MB/s 


In [0]:
import pandas as pd
link_hilstrom = 'https://drive.google.com/open?id=15osyN4c5z1pSo1JkxwL_N8bZTksRvQuU'
fluff, id = link_hilstrom.split('=')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Hillstrom.csv')
hillstrom_df = pd.read_csv('Hillstrom.csv')

In [0]:
import pandas as pd
link_ = 'https://drive.google.com/open?id=1b8N7WtwIe2WmQJD1KL5UAy70K13MxwKj'
fluff, id = link_.split('=')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Lalonde.csv')
lalonde_df = pd.read_csv('Lalonde.csv')

In [0]:
import csv
import json
import os
from os.path import isfile, join
from sklearn.model_selection import KFold, StratifiedKFold


def preprocess_data(df, dataset='hillstrom', verbose=True):
    """
    Preprocessing the dataset
     - Use one-hot encoding for categorical features
     - Check the name of the target variable and treatment variable
     - Drop the unused columns
     - Delete the unused data
    
    Args:
        df: A pandas.DataFrame which have all data of the dataset
        dataset: the name of the dataset
    Return:
        # I recommend to split into the dataframes of predictor variables, the 
        # target variable, and the treatment varaible
        # df_x: the dataframes of predictor variables
        # s_y: target variables
        # s_t: treatment variables
    """
    if dataset in ['hillstrom', 'email']:
        # For Hillstrom dataset, the ‘‘visit’’ target variable was selected
        #   as the target variable of interest and the selected treatment is 
        #   the e-mail campaign for women’s merchandise [1]
        # [1] Kane K, Lo VSY, Zheng J. True-lift modeling: Comparison of methods. 
        #    J Market Anal. 2014;2:218–238
    
        # Delete unused data: men's email cases should be removed
        df_x = df[df.segment != 'Mens E-Mail'].reset_index()
        # Assign Y for target (visit: 0, 1)
        s_y = df_x['visit']
        # Assign T for treatment (segment: Womens E-Mail, Mens E-Mail (not used), No E-Mail)
        s_t = (df_x['segment'] == 'Womens E-Mail').astype('int64')
        # Drop unused columns from X
        df_x = df_x.drop(columns=['conversion', 'spend', 'visit', 'segment', 'index'])
        # One-hot encoding for categorical features
        df_x = pd.get_dummies(df_x)

    elif dataset in ['criteo', 'ad']:
        raise NotImplementedError

    elif dataset in ['lalonde', 'job']:
        # Delete unused data: None
        df_x = df.reset_index()
        # Target variables (RE78: earnings in 1978)
        s_y = df_x['RE78']
        # Treatment variables (treatment: 0, 1)
        s_t = df_x['treatment']
        # Drop unused columns
        df_x = df_x.drop(columns=['treatment', 'RE78', 'index'])
        # One-hot encoding for categorical features
        df_x = pd.get_dummies(df_x)

    else:
        raise NotImplementedError

    return df_x, s_y, s_t

In [7]:
def print_preproccessing_data(df_x, s_y, s_t):
    print(df_x[:5])
    print(df_x.columns.values)
    print(df_x.shape)
    print(s_y[:5])
    print(s_t[:5])


hillstrom_df_x, hillstrom_s_y, hillstrom_s_t = preprocess_data(hillstrom_df, 'hillstrom')
print_preproccessing_data(hillstrom_df_x, hillstrom_s_y, hillstrom_s_t)

lalonde_df_x, lalonde_s_y, lalonde_s_t = preprocess_data(lalonde_df, 'lalonde')
print_preproccessing_data(lalonde_df_x, lalonde_s_y, lalonde_s_t)

"""
def print_preproccessing_data(df):
    print(df[:5])
    print(df.columns.values)
    print(df.shape)
    print(df['Y'][:5])
    print(df['T'][:5])


hillstrom_df_pre = preprocess_data(hillstrom_df, 'hillstrom')
print_preproccessing_data(hillstrom_df_pre)

lalonde_df_pre = preprocess_data(lalonde_df, 'lalonde')
print_preproccessing_data(lalonde_df_pre)
"""

   recency  history  mens  ...  channel_Multichannel  channel_Phone  channel_Web
0       10   142.44     1  ...                     0              1            0
1        6   329.08     1  ...                     0              0            1
2        7   180.65     0  ...                     0              0            1
3        2    45.34     1  ...                     0              0            1
4        6   134.83     0  ...                     0              1            0

[5 rows x 18 columns]
['recency' 'history' 'mens' 'womens' 'newbie'
 'history_segment_1) $0 - $100' 'history_segment_2) $100 - $200'
 'history_segment_3) $200 - $350' 'history_segment_4) $350 - $500'
 'history_segment_5) $500 - $750' 'history_segment_6) $750 - $1,000'
 'history_segment_7) $1,000 +' 'zip_code_Rural' 'zip_code_Surburban'
 'zip_code_Urban' 'channel_Multichannel' 'channel_Phone' 'channel_Web']
(42693, 18)
0    0
1    0
2    0
3    0
4    1
Name: visit, dtype: int64
0    1
1    0
2    1
3    1
4 

"\ndef print_preproccessing_data(df):\n    print(df[:5])\n    print(df.columns.values)\n    print(df.shape)\n    print(df['Y'][:5])\n    print(df['T'][:5])\n\n\nhillstrom_df_pre = preprocess_data(hillstrom_df, 'hillstrom')\nprint_preproccessing_data(hillstrom_df_pre)\n\nlalonde_df_pre = preprocess_data(lalonde_df, 'lalonde')\nprint_preproccessing_data(lalonde_df_pre)\n"

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

def performance(pr_y1_t1, pr_y1_t0, y, ct, groups=10):
    """
    1. Split the total customers into the given number of groups
    2. Calculate the statistics of each segment
    
    Args:
        pr_y1_ct1: the series (list) of the customer's expected return
        pr_y1_ct0: the expected return when a customer is not treated
        y: the observed return of customers
        ct: whether each customer is treated or not
        groups: the number of groups (segments). Should be 5, 10, or 20
    Return:
        DataFrame:
            columns:
                'n_y1_ct1': the number of treated responders
                'n_y1_ct0': the number of not treated responders
                'r_y1_ct1': the average return of treated customers
                'r_y1_ct0': the average return of not treated customers
                'n_ct1': the number of treated customers
                'n_ct0': the number of not treated customers
                'uplift': the average uplift (the average treatment effect)
            rows: the index of groups
    """

    ### check valid arguments
    if groups not in [5, 10, 20]:
        raise Exception("uplift: groups must be either 5, 10 or 20")

    ### check for NAs.
    if pr_y1_t1.isnull().values.any():
        raise Exception("uplift: NA not permitted in pr_y1_ct1")
    if pr_y1_t0.isnull().values.any():
        raise Exception("uplift: NA not permitted in pr_y1_ct0")
    if y.isnull().values.any():
        raise Exception("uplift: NA not permitted in y")
    if ct.isnull().values.any():
        raise Exception("uplift: NA not permitted in ct")

    ### check valid values for ct
    if set(ct) != {0, 1}:
        raise Exception("uplift: ct must be either 0 or 1")

    ### check length of arguments
    if not (len(pr_y1_t1) == len(pr_y1_t0) == len(y) == len(ct)):
        raise Exception("uplift: arguments pr_y1_ct1, pr_y1_ct0, y and ct must all have the same length")

    ###############################
    ###     Do it yourself!     ###
    ###############################
    # tr: treated responder, cr: controlled responder
    # Customers ordered by predicted uplift values in descending order are segmented.
    df = pd.DataFrame(data={'ct': ct, 'y': y,
                            'uplift_rank': (pr_y1_t1 - pr_y1_t0).rank(ascending=False, method='first')})
    df_group = df.groupby(pd.qcut(df['uplift_rank'], groups, labels=range(1, groups + 1)).rename('groups'))
    
    # Get group data
    n_ct1 = df_group['ct'].sum()
    n_ct0 = df_group.size() - n_ct1
    n_y1_t1 = df_group.apply(lambda r: r[r['ct'] == 1]['y'].sum())
    n_y1_t0 = df_group.apply(lambda r: r[r['ct'] == 0]['y'].sum())
    r_y1_t1 = n_y1_t1 / n_ct1
    r_y1_t0 = n_y1_t0 / n_ct0
    uplift = r_y1_t1 - r_y1_t0
    
    return pd.DataFrame(data={'n_y1_t1': n_y1_t1, 'n_y1_t0': n_y1_t0,
                              'r_y1_t1': r_y1_t1, 'r_y1_t0': r_y1_t0,
                              'n_t1': n_ct1, 'n_t0': n_ct0, 'uplift': uplift})


def qini(perf, plotit=True):
    nrow = len(perf)

    # Calculating the incremental gains. 
    # - First, the cumulitative sum of the treated and the control groups are
    #  calculated with respect to the total population in each group at the
    #  specified decile
    # - Afterwards we calculate the percentage of the total amount of people
    #  (both treatment and control) are present in each decile
    cumul_y1_t1 = (perf['n_y1_t1'].cumsum() / perf['n_t1'].cumsum()).fillna(0)
    cumul_y1_t0 = (perf['n_y1_t0'].cumsum() / perf['n_t0'].cumsum()).fillna(0)
    deciles = [i/nrow for i in range(1, nrow+1)]

    ### Model Incremental gains
    inc_gains = (cumul_y1_t1 - cumul_y1_t0) * deciles
    inc_gains = [0.0] + list(inc_gains)

    ### Overall incremental gains
    overall_inc_gain = sum(perf['n_y1_t1']) / sum(perf['n_t1']) \
            - sum(perf['n_y1_t0']) / sum(perf['n_t0'])

    ### Random incremental gains
    random_inc_gains = [i*overall_inc_gain / nrow for i in range(nrow+1)]

    ### Compute area under the model incremental gains (uplift) curve
    x = [0] + deciles
    y = list(inc_gains)
    auuc = 0
    auuc_rand = 0

    auuc_list = [auuc]
    for i in range(1, len(x)):
        auuc += 0.5 * (x[i] - x[i-1]) * (y[i] + y[i-1])
        auuc_list.append(auuc)

    ### Compute area under the random incremental gains curve
    y_rand = random_inc_gains

    auuc_rand_list = [auuc_rand]
    for i in range(1, len(x)):
        auuc_rand += 0.5 * (x[i] - x[i-1]) * (y_rand[i] + y_rand[i-1])
        auuc_rand_list.append(auuc_rand)

    ### Compute the difference between the areas (Qini coefficient)
    Qini = auuc - auuc_rand

    ### Plot incremental gains curve
    if plotit:
        x_axis = x
        plt.plot(x_axis, inc_gains)
        plt.plot(x_axis, random_inc_gains)
        plt.show()
    
    ### Qini 30%, Qini 10%
    n_30p = int(nrow*3/10)
    n_10p = int(nrow/10)
    qini_30p = auuc_list[n_30p] - auuc_rand_list[n_30p]
    qini_10p = auuc_list[n_10p] - auuc_rand_list[n_10p]

    res = {
        'qini': Qini,
        'inc_gains': inc_gains,
        'random_inc_gains': random_inc_gains,
        'auuc_list': auuc_list,
        'auuc_rand_list': auuc_rand_list,
        'qini_30p': qini_30p,
        'qini_10p': qini_10p,
    }    

    return res


def qini2(perf, plotit=True):
    """
    Calculating the incremental gains (y-axis of Qini curve)
     - First, the cumulitative sum of the treated and the control groups are
      calculated with respect to the total population in each group at the
      specified decile
     - Afterwards we calculate the percentage of the total amount of people
      (both treatment and control) are present in each decile
    Args:
        perf: A return of the performance function (above)
        plotit: whether draw a plot or not
    Return:
        1. Qini value
        2. return or save the plot if plotit is True
    """
    
    ###############################
    ###     Do it yourself!     ###
    ###############################
    # Get qini value
    s_rt = perf['r_y1_ct1'].cumsum() / perf['n_y1_ct1'].cumsum()
    s_rc = perf['r_y1_ct0'].cumsum() / perf['n_y1_ct0'].cumsum()
    k = len(s_rt)
    qini_value = s_rt * s_rt.index / k

    # Draw qini curve
    plt.rcParams['figure.figsize'] = (5.0, 4.0)
    plt.plot(qini_value, label='qini')
    plt.legend(['Bad model'])
    plt.show()
    
    return qini_value


In [10]:
import itertools
import numpy as np

def parameter_tuning(fit_mdl, pred_mdl, data, search_space):
    """
    Given a model, search all combination of parameter sets and find
    the best parameter set
    
    Args:
        fit_mdl: model function
        pred_mdl: predict function of fit_mdl
        data:
            {
                "x_train": predictor variables of training dataset,
                "y_train": target variables of training dataset,
                "ct_train": treatment variables of training dataset,
                "x_test": predictor variables of test (usually, validation) dataset,
                "y_test": target variables of test (usually, validation) dataset,
                "ct_test": treatment variables of test (usually, validation) dataset,
            }
        search_space:
            {
                parameter_name: [search values]
            }
    Return:
        The best parameter set
    """
    
    ###############################
    ###     Do it yourself!     ###
    ###############################
    return ##

  
def wrapper(fit_mdl, pred_mdl, data)
    """
    General wrapper approach
    
    Args:
        fit_mdl: model function
        pred_mdl: predict function of fit_mdl
        data:
            {
                "x_train": predictor variables of training dataset,
                "y_train": target variables of training dataset,
                "ct_train": treatment variables of training dataset,
                "x_test": predictor variables of test (usually, validation) dataset,
                "y_test": target variables of test (usually, validation) dataset,
                "ct_test": treatment variables of test (usually, validation) dataset,
            }
    Return:
        (A list of best models, The list of dropped variables)
    """
    
    ###############################
    ###     Do it yourself!     ###
    ###############################
    return ##


SyntaxError: ignored

In [0]:
import itertools
import numpy as np

def parameter_tuning(fit_mdl, pred_mdl, data, search_space):
    """
    Given a model, search all combination of parameter sets and find
    the best parameter set
    
    Args:
        fit_mdl: model function
        pred_mdl: predict function of fit_mdl
        data:
            {
                "x_train": predictor variables of training dataset,
                "y_train": target variables of training dataset,
                "t_train": treatment variables of training dataset,
                "x_test": predictor variables of test (usually, validation) dataset,
                "y_test": target variables of test (usually, validation) dataset,
                "t_test": treatment variables of test (usually, validation) dataset,
            }
        search_space:
            {
                parameter_name: [search values]
            }
    Return:
        The best parameter set
    """
    x_train = data['x_train']
    y_train = data['y_train']
    t_train = data['t_train']
    x_test = data['x_test']
    y_test = data['y_test']
    t_test = data['t_test']
    
    max_q = -float('inf')
    best_mdl = None

    keys = search_space.keys()
    n_space = [len(search_space[key]) for key in keys]
    n_iter = np.prod(n_space)
    
    best_params = None
    for i in range(n_iter):
        params = {}
        for idx, key in enumerate(keys):
            params[key] = search_space[key][i % n_space[idx]]
            i = int(i / n_space[idx])

        mdl = fit_mdl(x_train, y_train, t_train, **params)
        pred = pred_mdl(mdl, newdata=x_test, y=y_test, ct=t_test)
        # print('    {}'.format(params))
        try:
            perf = performance(pred['pr_y1_t1'], pred['pr_y1_t0'], y_test, t_test)
        except Exception as e:
            print(e)
            continue
        q = qini(perf, plotit=False)['qini']
        if q > max_q:
            max_q = q
            best_mdl = mdl
            best_params = params

    return best_mdl, best_params


def wrapper(fit_mdl, pred_mdl, data, params=None,
            best_models=None, drop_variables=None, qini_values=None):
    """
    General wrapper approach
    
    Args:
        fit_mdl: model function
        pred_mdl: predict function of fit_mdl
        data:
            {
                "x_train": predictor variables of training dataset,
                "y_train": target variables of training dataset,
                "t_train": treatment variables of training dataset,
                "x_test": predictor variables of test (usually, validation) dataset,
                "y_test": target variables of test (usually, validation) dataset,
                "t_test": treatment variables of test (usually, validation) dataset,
            }
    Return:
        (A list of best models, The list of dropped variables)
    """
    if best_models is None:
        best_models = []
    if drop_variables is None:
        drop_variables = []
    if qini_values is None:
        qini_values = []
    if params is None:
        params = {}

    x_train = data['x_train']
    y_train = data['y_train']
    t_train = data['t_train']
    x_test = data['x_test']
    y_test = data['y_test']
    t_test = data['t_test']

    variables = data['x_train'].columns

    max_q = -float('inf')
    drop_var = None
    best_mdl = None
    for var in variables:
        if var in drop_variables:
            continue
        x = x_train.copy()
        x.drop(drop_variables + [var], axis=1, inplace=True)
        mdl = fit_mdl(x, y_train, t_train, **params)
        x = x_test.copy()
        x.drop(drop_variables + [var], axis=1, inplace=True)
        pred = pred_mdl(mdl, newdata=x, y=y_test, ct=t_test)
        perf = performance(pred['pr_y1_t1'], pred['pr_y1_t0'], y_test, t_test)
        q = qini(perf, plotit=False)['qini']
        if q > max_q:
            max_q = q
            drop_var = var
            best_mdl = mdl
    
    best_models.append(best_mdl)
    drop_variables.append(drop_var)
    qini_values.append(max_q)

    left_vars = [var for var in variables if (var not in drop_variables)]
    
    if len(variables) == len(drop_variables) + 1:
        return best_models, drop_variables + left_vars, qini_values
    else:
        return wrapper(fit_mdl, pred_mdl, data, params=params,
                       best_models=best_models, drop_variables=drop_variables,
                       qini_values=qini_values)

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression


def tma(x, y, ct, method=LogisticRegression, **kwargs):
    """Training a model according to the "Two Model Approach" 
    (a.k.a. "Separate Model Approach")
    The default model is General Linear Model (GLM)
    
    Source: "Incremental Value Modeling" (Hansotia, 2002)

    Args:
        x: A data frame of predictors.
        y: A binary response (numeric) vector.
        ct: A binary response (numeric) representing the treatment assignment
            (coded as 0/1).
        method: A sklearn model specifying which classification or regression
            model to use. This should be a method that can handle a 
            multinominal class variable.

    Return:
        Dictionary: A dictionary of two models. One for the treatment group, 
            one for the control group.

            {
                'model_treat': a model for the treatment group,
                'model_control': a model for the control group
            }

    """
    
    ###############################
    ###     Do it yourself!     ###
    ###############################
    return {
        'model_treat': method(**kwargs).fit(x[ct == 1], y[ct == 1]),
        'model_control': method(**kwargs).fit(x[ct == 0], y[ct == 0])
    }


def predict_tma(obj, newdata, **kwargs):
    """Predictions according to the "Two Model Approach" 
    (a.k.a. "Separate Model Approach")
    
    For each instance in newdata two predictions are made:
    1) What is the probability of a person responding when treated?
    2) What is the probability of a person responding when not treated
      (i.e. part of control group)?

    Source: "Incremental Value Modeling" (Hansotia, 2002)

    Args:
        obj: A dictionary of two models. 
            One for the treatment group, one for the control group.
        newdata: A data frame containing the values at which predictions
            are required.
    
    Return:
        DataFrame: A dataframe with predicted returns for when the customers
            are treated and for when they are not treated.
            'pr_y1_ct1': when treated, 'pr_y1_ct0': when not treated
    """

    ###############################
    ###     Do it yourself!     ###
    ###############################
    # LogisticRegression: use predict_proba and return result[:, 1] for True class
    # LinearRegression: use predict
    if isinstance(obj['model_treat'], LogisticRegression):
        return pd.DataFrame(data={
            'pr_y1_t1': obj['model_treat'].predict_proba(newdata)[:, 1],
            'pr_y1_t0': obj['model_control'].predict_proba(newdata)[: , 1]
        })
    else:
        return pd.DataFrame(data={
            'pr_y1_t1': obj['model_treat'].predict(newdata),
            'pr_y1_t0': obj['model_control'].predict(newdata)
        })

In [0]:
import sys
import math


class Node(object):
    def __init__(self, attribute, threshold):
        self.attr = attribute
        self.thres = threshold
        self.left = None
        self.right = None
        self.leaf = False
        self.predict = None


def select_threshold(df, attribute, predict_attr):
    """
    Select the threshold of the attribute to split
    The threshold chosen splits the test data such that information gain is maximized
    """
    # Convert dataframe column to a list and round each value
    values = df[attribute].tolist()
    values = [float(x) for x in values]
    # Remove duplicate values by converting the list to a set, then sort the set
    values = set(values)
    values = list(values)
    values.sort()
    max_ig = float("-inf")
    thres_val = 0
    # try all threshold values that are half-way between successive values in this sorted list
    for i in range(0, len(values) - 1):
        thres = (values[i] + values[i+1])/2
        ig = info_gain(df, attribute, predict_attr, thres)
        if ig > max_ig:
            max_ig = ig
            thres_val = thres
    # Return the threshold value that maximizes information gained
    return thres_val


def info_entropy(df, predict_attr):
    """
    Calculate info content (entropy) of the test data
    """
    # Dataframe and number of positive/negatives examples in the data
    p_df = df[df[predict_attr] == 1]
    n_df = df[df[predict_attr] == 0]
    p = float(p_df.shape[0])
    n = float(n_df.shape[0])
    # Calculate entropy
    if p  == 0 or n == 0:
        I = 0
    else:
        I = ((-1*p)/(p + n))*math.log(p/(p+n), 2) + ((-1*n)/(p + n))*math.log(n/(p+n), 2)
    return I


def remainder(df, df_subsets, predict_attr):
    """
    Calculates the weighted average of the entropy after an attribute test
    """
    # number of test data
    num_data = df.shape[0]
    remainder = float(0)
    for df_sub in df_subsets:
        if df_sub.shape[0] > 1:
            remainder += float(df_sub.shape[0]/num_data)*info_entropy(df_sub, predict_attr)
    return remainder


def info_gain(df, attribute, predict_attr, threshold):
    """
    Calculates the information gain from the attribute test based on a given threshold
    Note: thresholds can change for the same attribute over time
    """
    sub_1 = df[df[attribute] <= threshold]
    sub_2 = df[df[attribute] > threshold]
    # Determine information content, and subract remainder of attributes from it
    ig = info_entropy(df, predict_attr) - remainder(df, [sub_1, sub_2], predict_attr)
    return ig


def num_class(df, predict_attr):
    """
    Returns the number of positive and negative data
    """
    p_df = df[df[predict_attr] == 1]
    n_df = df[df[predict_attr] == 0]
    return p_df.shape[0], n_df.shape[0]


def choose_attr(df, attributes, predict_attr):
    """
    Chooses the attribute and its threshold with the highest info gain
    from the set of attributes
    """
    max_info_gain = float("-inf")
    best_attr = None
    threshold = 0
    # Test each attribute (note attributes maybe be chosen more than once)
    for attr in attributes:
        thres = select_threshold(df, attr, predict_attr)
        ig = info_gain(df, attr, predict_attr, thres)
        if ig > max_info_gain:
            max_info_gain = ig
            best_attr = attr
            threshold = thres
    return best_attr, threshold



def build_tree(df, cols, predict_attr):
    """
    Builds the Decision Tree based on training data, attributes to train on,
    and a prediction attribute
    """
    # Get the number of positive and negative examples in the training data
    p, n = num_class(df, predict_attr)
    # If train data has all positive or all negative values or less than the
    # given minimum number of split. Then we have reached the end of our tree
    if p == 0 or n == 0 or (p+n) < 100:
        # Create a leaf node indicating it's prediction
        leaf = Node(None,None)
        leaf.leaf = True
        leaf.predict = p / (p+n)
        return leaf
    else:
        # Determine attribute and its threshold value with the highest
        # information gain
        best_attr, threshold = choose_attr(df, cols, predict_attr)
        # Create internal tree node based on attribute and it's threshold
        tree = Node(best_attr, threshold)
        sub_1 = df[df[best_attr] <= threshold]
        sub_2 = df[df[best_attr] > threshold]
        # Recursively build left and right subtree
        tree.left = build_tree(sub_1, cols, predict_attr)
        tree.right = build_tree(sub_2, cols, predict_attr)
        return tree


def predict(node, row_df):
    """
    Given a instance of a training data, make a prediction of an observation (row)
    based on the Decision Tree
    Assumes all data has been cleaned (i.e. no NULL data)
    """
    # If we are at a leaf node, return the prediction of the leaf node
    if node.leaf:
        return node.predict
    # Traverse left or right subtree based on instance's data
    if row_df[node.attr] <= node.thres:
        return predict(node.left, row_df)
    elif row_df[node.attr] > node.thres:
        return predict(node.right, row_df)


def test_predictions(root, df, target_attr='y'):
    """
    Given a set of data, make a prediction for each instance using the Decision Tree
    """
    prediction = []
    for index,row in df.iterrows():
        prediction.append(predict(root, row))
    pred_df = pd.Series(prediction)
    return pred_df

In [215]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

def main():
    ### Hyper parameters ###
    dataset = 'hillstrom'
    seed = 1234
    n_fold = 5
    models = {
        'tma': {'model': tma, 'predict': predict_tma}
    }
    search_space = {
        'method': [LogisticRegression],
        'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
        'penalty': ['none', 'l2'],
        'tol': [1e-2, 1e-3, 1e-4],
        'C': [1e6, 1e3, 1, 1e-3, 1e-6]
    }

    ### Load data & K fold validation ###
    if dataset == 'hillstrom':
        df_x, s_y, s_t = preprocess_data(hillstrom_df, dataset)
        fold_split = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed).split(df_x, s_y)
    elif dataset == 'lalonde':
        df_x, s_y, s_t = preprocess_data(lalonde_df, dataset)
        fold_split = KFold(n_splits=n_fold, shuffle=True, random_state=seed).split(df_x)
    else:
        assert()

    for model_name in models:
        qinis = []
        
        for train_index, test_index in fold_split:
            x_train = df_x.reindex(train_index).reset_index(drop=True)
            y_train = s_y.reindex(train_index).reset_index(drop=True)
            t_train = s_t.reindex(train_index).reset_index(drop=True)
            
            x_test = df_x.reindex(test_index).reset_index(drop=True)
            t_test = s_t.reindex(test_index).reset_index(drop=True)
            y_test = s_y.reindex(test_index).reset_index(drop=True)
            """
            df_train = x_train
            df_train['Y'] = y_train
            df_train['T'] = t_train
            
            df_test = x_test
            df_test['Y'] = y_test
            df_test['T'] = t_test
            
            stratify = pd.DataFrame([y_train, t_train]).T
            df_tune, df_val = train_test_split(df_train, test_size=0.33, random_state=1234, stratify=stratify)

            data_dict = {
                "x_train": df_tune.drop(columns=['Y', 'T']).reset_index(drop=True),
                "y_train": df_tune['Y'].reset_index(drop=True),
                "t_train": df_tune['T'].reset_index(drop=True),
                "x_test": df_val.drop(columns=['Y', 'T']).reset_index(drop=True),
                "y_test": df_val['Y'].reset_index(drop=True),
                "t_test": df_val['T'].reset_index(drop=True),
            }

            model_method = search_space.get('method', None)
            params = {
                'method': None if model_method is None else model_method[0],
            }
            if params['method'] == LogisticRegression:
                solver = search_space.get('solver', None)
                params['solver'] = None if solver is None else solver[0]

            _, drop_vars, qini_values = wrapper(tma, predict_tma, data_dict, params=params)
            
            best_qini = max(qini_values)
            best_idx = qini_values.index(best_qini)
            best_drop_vars = drop_vars[:best_idx]

            #x_tune.drop(best_drop_vars, axis=1, inplace=True)
            #x_val.drop(best_drop_vars, axis=1, inplace=True)
            #x_train.drop(best_drop_vars, axis=1, inplace=True)
            #x_test.drop(best_drop_vars, axis=1, inplace=True)

            _, best_params = parameter_tuning(tma, predict_tma, data_dict, search_space=search_space)           
            """

            ### CODE for decision tree ###
            # An example use of 'build_tree' and 'predict'
            df_train = x_train.copy()
            df_train['Y'] = y_train
            df_train['T'] = t_train
            df_test = x_test.copy()
            df_test['Y'] = y_test
            df_test['T'] = y_test

            # Drop history columne due to too slow learning speed
            df_train.drop(['history'], axis=1, inplace=True)
            df_test.drop(['history'], axis=1, inplace=True)

            assert((df_train.columns == df_test.columns).all())
            attributes = [c for c in df_train.columns if c != 'Y']
            root = build_tree(df_train, attributes, 'Y')
            pred = test_predictions(root, df_test, target_attr='Y')
            print('pred: {}'.format(pred))
            continue
            ### CODE for decision tree ###

            model = models[model_name]['model'](x_train, y_train, t_train)
            pred = models[model_name]['predict'](model, x_test)
            # print(pred[: 5])
            
            perf = performance(pred['pr_y1_t1'], pred['pr_y1_t0'], y_test, t_test)
            print(perf[:5])
            
            q = qini(perf)
            print(q)
            qinis.append(q['qini'])

            
        ### Variable selection (General wrapper approach) ###

        ### Parameter tuning ###
        print("Model: {}\n".format(model_name))
        # print("Tuning space: \n")
        # for key, val in search_space.items():
        #    print("    '{}': {}\n".format(key, val))
        print("Seed: {}\n".format(seed))
        print('Qini values: ', qinis)
        print("Qini value: mean = {}, std = {}\n\n".format(np.mean(qinis), np.std(qinis)))
        
main()


pred: 0       0.000000
1       0.098039
2       0.164706
3       0.000000
4       0.094595
5       0.187500
6       0.102041
7       0.000000
8       0.101695
9       0.074627
10      0.184615
11      0.100000
12      0.323529
13      0.181818
14      0.175000
15      0.082192
16      0.184783
17      0.148936
18      0.220779
19      0.135135
20      0.052632
21      0.209302
22      0.062500
23      0.078431
24      0.123457
25      0.113402
26      0.052632
27      0.178571
28      0.127907
29      0.186047
          ...   
8509    0.059524
8510    0.164179
8511    0.084746
8512    0.184783
8513    0.139535
8514    0.037037
8515    0.073171
8516    0.181818
8517    0.074074
8518    0.113636
8519    0.102041
8520    0.115385
8521    0.000000
8522    0.000000
8523    0.080000
8524    0.112676
8525    0.000000
8526    0.049383
8527    0.085714
8528    0.056818
8529    0.200000
8530    0.011236
8531    0.000000
8532    0.327869
8533    0.114286
8534    0.204082
8535    0.088889
8536    

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
